# Test to see if I can make a very large zarr store and populate it section at a time

* Idea: take a .nc file (converted from a .pp file) and use to make a giant zarr file for over a year. Then use the .nc file to populate a small section of it (5 hours).
* Note, you need to make sure you chunk the orig file or you will quickly run out of memory. 

In [10]:
from pathlib import Path

import dask.array
import iris
import pandas as pd
import xarray as xr

In [5]:
basedir = Path('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp')
streams = sorted(basedir.glob('apver*.pp'))
streams

[PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apvera.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverb.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverc.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverd.pp')]

In [6]:
tmpdir = Path('/work/scratch-nopw2/mmuetz/wcrp_hackathon/tmp_iris_conversion')

In [7]:
ncoutputs = []
all_cubes = {}
for s in streams:
    pp = sorted(s.glob('*.pp'))[0]
    ncout = (tmpdir / pp.name).with_suffix('.nc')
    ncoutputs.append(ncout)


In [11]:
dummies = dask.array.zeros((366 * 8, 25, 3840, 5120), chunks=(1, 5, 640, 640))

In [12]:
dummies.shape

(2928, 25, 3840, 5120)

In [36]:
time = pd.date_range('2020-1-1', '2020-12-31 23:00', freq='3h')

In [8]:
ds = xr.open_dataset(ncoutputs[2], chunks={'time': 1, 'pressure': 5, 'latitude': 640, 'longitude': 640})

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_5096/3901118129.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset(ncoutputs[2], chunks={'time': 1, 'pressure': 5, 'latitude': 640, 'longitude': 640})


In [38]:

ds
coords = dict(
    time=time,
    pressure=ds.pressure.drop_vars('forecast_reference_time'),
    latitude=ds.latitude.drop_vars('forecast_reference_time'),
    longitude=ds.longitude.drop_vars('forecast_reference_time'),
)

In [48]:
dsout = xr.Dataset(
    {
        'upward_air_velocity': (('time', 'pressure', 'latitude', 'longitude'), dummies), 
        'air_temperature': (('time', 'pressure', 'latitude', 'longitude'), dummies), 
    }, 
    coords=coords,
)

In [41]:
ds2 = xr.open_dataset(ncoutputs[3], chunks={'time': 1, 'pressure': 5, 'latitude': 640, 'longitude': 640})

/tmp/ipykernel_5096/1187667236.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds2 = xr.open_dataset(ncoutputs[3], chunks={'time': 1, 'pressure': 5, 'latitude': 640, 'longitude': 640})


In [49]:
zarr_path = '/work/scratch-nopw2/mmuetz/wcrp_hackathon/tmp_iris_conversion/giant.zarr'

In [50]:
dsout.to_zarr(zarr_path, compute=False)

Delayed('_finalize_store-bc8bb236-5d0c-4e6d-b61a-ba917956fe10')

In [44]:
w = ds.upward_air_velocity

In [45]:
w = w.drop_vars(['forecast_period', 'forecast_reference_time'])

In [51]:
T = ds2.air_temperature.drop_vars(['forecast_period', 'forecast_reference_time'])

In [52]:
w

<xarray.DataArray 'upward_air_velocity' (time: 5, pressure: 25, latitude: 3840,
                                         longitude: 5120)> Size: 10GB
dask.array<open_dataset-upward_air_velocity, shape=(5, 25, 3840, 5120), dtype=float32, chunksize=(1, 5, 640, 640), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 40B 2020-01-20 ... 2020-01-20T12:00:00
  * latitude   (latitude) float32 15kB -89.98 -89.93 -89.88 ... 89.93 89.98
  * longitude  (longitude) float32 20kB 0.03516 0.1055 0.1758 ... 359.9 360.0
  * pressure   (pressure) float32 100B 1.0 5.0 10.0 20.0 ... 950.0 975.0 1e+03
Attributes:
    standard_name:    upward_air_velocity
    units:            m s-1
    um_stash_source:  m01s15i242
    grid_mapping:     latitude_longitude

In [53]:
T

<xarray.DataArray 'air_temperature' (time: 5, pressure: 25, latitude: 3840,
                                     longitude: 5120)> Size: 10GB
dask.array<open_dataset-air_temperature, shape=(5, 25, 3840, 5120), dtype=float32, chunksize=(1, 5, 640, 640), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 40B 2020-01-20 ... 2020-01-20T12:00:00
  * pressure   (pressure) float32 100B 1.0 5.0 10.0 20.0 ... 950.0 975.0 1e+03
  * latitude   (latitude) float32 15kB -89.98 -89.93 -89.88 ... 89.93 89.98
  * longitude  (longitude) float32 20kB 0.03516 0.1055 0.1758 ... 359.9 360.0
Attributes:
    standard_name:    air_temperature
    units:            K
    um_stash_source:  m01s16i203
    grid_mapping:     latitude_longitude

In [54]:
w.time.values == T.time.values

array([ True,  True,  True,  True,  True])

In [13]:
#f1 = w.rename('f1')

In [29]:
import numpy as np
np.argmin(np.abs(w.time.values[0] - w2.time.values))

np.int64(152)

In [27]:
w.time.values[0]

np.datetime64('2020-01-20T00:00:00.000000000')

In [30]:
w2.time.values[152]

np.datetime64('2020-01-20T00:00:00.000000000')

In [31]:
w.time.values == w2.time.values[152:157]

array([ True,  True,  True,  True,  True])

In [55]:
T.to_zarr(zarr_path, region={'time': slice(152, 157), 'pressure': slice(None), 'latitude': slice(None), 'longitude': slice(None)})

In [56]:
w.to_zarr(zarr_path, region={'time': slice(152, 157), 'pressure': slice(None), 'latitude': slice(None), 'longitude': slice(None)})

In [57]:
ds_zarr = xr.open_zarr(zarr_path)

In [58]:
ds_zarr

<xarray.Dataset> Size: 23TB
Dimensions:              (time: 2928, pressure: 25, latitude: 3840,
                          longitude: 5120)
Coordinates:
  * latitude             (latitude) float32 15kB -89.98 -89.93 ... 89.93 89.98
  * longitude            (longitude) float32 20kB 0.03516 0.1055 ... 359.9 360.0
  * pressure             (pressure) float32 100B 1.0 5.0 10.0 ... 975.0 1e+03
  * time                 (time) datetime64[ns] 23kB 2020-01-01 ... 2020-12-31...
Data variables:
    air_temperature      (time, pressure, latitude, longitude) float64 12TB dask.array<chunksize=(1, 5, 640, 640), meta=np.ndarray>
    upward_air_velocity  (time, pressure, latitude, longitude) float64 12TB dask.array<chunksize=(1, 5, 640, 640), meta=np.ndarray>

In [59]:
ds_zarr.isel(time=slice(150, 160)).upward_air_velocity.mean(dim=['pressure', 'latitude', 'longitude']).compute()

<xarray.DataArray 'upward_air_velocity' (time: 10)> Size: 80B
array([        nan,         nan, -0.00037511, -0.00056147, -0.00038544,
       -0.00027387, -0.00024242,         nan,         nan,         nan])
Coordinates:
  * time     (time) datetime64[ns] 80B 2020-01-19T18:00:00 ... 2020-01-20T21:...

In [60]:
ds_zarr.isel(time=slice(150, 160)).air_temperature.mean(dim=['pressure', 'latitude', 'longitude']).compute()

<xarray.DataArray 'air_temperature' (time: 10)> Size: 80B
array([         nan,          nan, 247.74833009, 247.71671881,
       247.7165363 , 247.7471753 , 247.77932349,          nan,
                nan,          nan])
Coordinates:
  * time     (time) datetime64[ns] 80B 2020-01-19T18:00:00 ... 2020-01-20T21:...

In [20]:
w2.time

<xarray.DataArray 'time' (time: 2928)> Size: 23kB
array(['2020-01-01T00:00:00.000000000', '2020-01-01T03:00:00.000000000',
       '2020-01-01T06:00:00.000000000', ..., '2020-12-31T15:00:00.000000000',
       '2020-12-31T18:00:00.000000000', '2020-12-31T21:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 23kB 2020-01-01 ... 2020-12-31T21:00:00